In [1]:
def rename_from_fn_to_slice_numbers( image_list, sorted_fn_list, testing=True):
    # RENAME FROM FILENAMES TO SLICE NAMES
    for image_full_name in image_list:
        for sorted_fn_line in sorted_fn_list:
            fn, slice_num = sorted_fn_line.split(' ')
            if fn=='Placeholder':
                print(slice_num)

            if fn in image_full_name:
                # Slice number need to be preceeded by zeros to be 3 digits
                if len(slice_num)==1:
                    slice_num = '00'+slice_num
                elif len(slice_num)==2:
                    slice_num = '0'+slice_num

                old_designation = image_path+image_full_name
                new_designation = image_path+stack+'_slice_'+slice_num+'.jpg'
                print(old_designation)
                print(new_designation)
                
                if not testing:
                    ! cp $old_designation $new_designation


def insert_dummy_placeholder_images( image_list, sorted_fn_list, testing=True):
    # FILL IN PLACEHOLDER IMAGES WITH THE PREVIOUS IMAGE RENAMED
    prev_slice_num = -0.1

    for image_full_name in image_list:
        img_slice_num = image_full_name.split('_')[2][:image_full_name.split('_')[2].index('.')]

        # Iterate through every fn/slice number in sorted filenames
        for sorted_fn_line in sorted_fn_list:
            fn_sfns, slice_num_sfns = sorted_fn_line.split(' ')
            if len(slice_num_sfns)==1:
                slice_num_sfns = '00'+slice_num_sfns
            elif len(slice_num_sfns)==2:
                slice_num_sfns = '0'+slice_num_sfns

            # If there is a placeholder slice after the current img_slice_num, then copy
            if int(slice_num_sfns)==int(img_slice_num)+1 and fn_sfns=='Placeholder':
                prev_img_fn = stack+'_slice_'+str(img_slice_num)+'.jpg'
                placeholder_img_fn = stack+'_slice_'+slice_num_sfns+'_placeholder_cfp.jpg'
                print(prev_img_fn)
                print(placeholder_img_fn)
                
                if not testing:
                    ! cp $images_local/$prev_img_fn $images_local/$placeholder_img_fn
                    
import os
import sys
import json
from shutil import copyfile

sys.path.append(os.path.join( os.environ['NG_REPO_DIR'], 'src', 'utilities'))
from filepath_manager import *
from precomputing_utilities import *
from imported_atlas_utilities import *

atlas_root = os.environ['ATLAS_DATA_ROOT_DIR']

In [3]:
load_sorted_filenames_from_fp( '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_sorted_filenames.txt' )

[{'DK23_slide001_2019_09_10_S05_S4_C0_CFP': 2,
  'DK23_slide001_2019_09_10_S09_S8_C0_CFP': 3,
  'DK23_slide002_2019_09_10_S01_S0_C0_CFP': 4,
  'DK23_slide002_2019_09_10_S06_S5_C0_CFP': 5,
  'DK23_slide002_2019_09_10_S11_S10_C0_CFP': 6,
  'DK23_slide002_2019_09_10_S16_S15_C0_CFP': 7,
  'DK23_slide003_2019_09_10_S01_S0_C0_CFP': 8,
  'DK23_slide003_2019_09_10_S06_S5_C0_CFP': 9,
  'DK23_slide003_2019_09_10_S11_S10_C0_CFP': 10,
  'DK23_slide004_2019_09_10_S01_S0_C0_CFP': 12,
  'DK23_slide004_2019_09_10_S06_S5_C0_CFP': 13,
  'DK23_slide004_2019_09_10_S11_S10_C0_CFP': 14,
  'DK23_slide004_2019_09_10_S16_S15_C0_CFP': 15,
  'DK23_slide005_2019_09_10_S01_S0_C0_CFP': 16,
  'DK23_slide005_2019_09_10_S06_S5_C0_CFP': 17,
  'DK23_slide005_2019_09_10_S11_S10_C0_CFP': 18,
  'DK23_slide005_2019_09_10_S17_S16_C0_CFP': 19,
  'DK23_slide006_2019_09_10_S01_S0_C0_CFP': 20,
  'DK23_slide006_2019_09_10_S06_S5_C0_CFP': 21,
  'DK23_slide006_2019_09_10_S12_S11_C0_CFP': 22,
  'DK23_slide006_2019_09_10_S18_S17_C0_C

In [1]:
def rename_all_input_images(input_fp, output_fp, sorted_fns_fp):
    input_imgs = os.listdir( input_fp )
    # fns = keys, sections = vals
    sorted_fns = load_sorted_filenames_from_fp( sorted_fns_fp )[0]
    
    # Rename all files in "input_fp"
    for input_img_fn in input_imgs:
        extension = input_img_fn.split('.')[1]
        for fn in sorted_fns.keys():
            channel_neutral_fn = fn
            try:
                channel_neutral_fn = fn[:fn.index('_C')]
            except:
                pass
            # If the image matches the sorted_fns filename, copy and rename
            if channel_neutral_fn in input_img_fn:
                curr_fn = os.path.join( input_fp, input_img_fn )
                new_fn = os.path.join( output_fp, str(sorted_fns[fn])+'.'+extension )
                # Copy the file
                print('cp '+curr_fn+' '+new_fn)
                copyfile(curr_fn, new_fn)
                break

def create_precomputation_data_file():
    """
    This function will take the input and output filepaths or the CZI->TIFF converter.
    """
    # Filepaths to image files already on the computer
    #    NOT NECESSARY as long as the image files are in the proper location
    #    (check using `get_image_fp()`)
    atlas_raw_jpeg_image_path = atlas_root+'CSHL_data_processed/'+stack+'/'+stack+'_prep2_raw_grayJpeg/'
    atlas_sorted_fn_path = atlas_root+'CSHL_data_processed/'+stack+'/'+stack+'_sorted_filenames.txt'
    
    # Input image files for precomputing script
    temp_renamed_image_path = get_image_fp( stack, precomputed=False, brain_crop='brainstem', \
                                         image_version='grayJpeg', resolution='raw' )
    precomputed_input_image_path = get_image_fp( stack, precomputed=False, brain_crop='brainstem', \
                                         image_version='grayJpeg', resolution='raw' )
    temp_sorted_fn_path = precomputed_input_image_path+'../'+stack+'_sorted_filenames.txt'

    # Output files from precomputing script
    precomputed_output_image_path = get_image_fp( stack, precomputed=True, brain_crop='brainstem', \
                                         image_version='grayJpeg', resolution='raw' )
    
    # sorted_filename
    # precomputed_input_image_path
    # ext
    data = {
        "s3_creds_file": "/home/alexn/Documents/json_credentials/alex_aws_credentials.json",
        "s3_bucket_name_for_download": "mousebrainatlas-data",
        #"sorted_filename": stack+"_sorted_filenames.txt",
        "ext": ".jpg",
        "folder_to_write_to": "/mnt/x/temp_s3_download/"+stack+"_prep2_lossless_grayJpeg",
        "folder_to_convert_from": precomputed_input_image_path,
        "folder_to_convert_to": precomputed_output_image_path,
        "s3_bucket_name_for_upload": "NULL",
        "s3_dir_to_write_to": "precomputed/"+stack+"_fullres",
        "voxel_resolution": [460, 460, 20000],
        "voxel_offset": [0, 0, 0],
        "overwrite": True
    }
    
    with open('input.json', 'w') as file:
        json.dump(data, file)

In [18]:
rfp = '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/'

rename_all_input_images(input_fp=rfp+'DK23_prep1_thumbnail_C0', 
                        output_fp=rfp+'DK23_Neuroglancer_secondary_channel_thumbnail', 
                        sorted_fns_fp=rfp+'DK23_sorted_filenames.txt')

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide002_2019_09_10_S11_S10_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/6.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide075_2019_09_11_S15_S14_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/275.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide005_2019_09_10_S17_S16_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/19.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide105_2019_09_10_S13_S12_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide063_2019_09_11_S15_S14_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/233.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide089_2019_09_10_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/324.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide112_2019_09_11_S19_S18_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/419.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide106_2019_09_10_S19_S18_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbna

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide099_2019_09_10_S08_S7_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/365.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide116_2019_09_10_S07_S6_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/433.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide007_2019_09_10_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/24.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide090_2019_09_10_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/3

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide113_2019_09_10_S13_S12_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/422.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide010_2019_09_10_S07_S6_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/36.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide020_2019_09_11_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/73.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide038_2019_09_10_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/1

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide029_2019_09_10_S08_S7_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/109.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide066_2019_09_11_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/243.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide035_2019_09_10_S15_S14_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/128.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide104_2019_09_10_S19_S18_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnai

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide064_2019_09_11_S22_S21_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/238.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide074_2019_09_11_S15_S14_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/272.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide017_2019_09_11_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/62.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide013_2019_09_10_S07_S6_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide041_2019_09_11_S08_S7_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/146.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide037_2019_09_10_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/133.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide048_2019_09_11_S08_S7_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/173.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide052_2019_09_11_S22_S21_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide090_2019_09_10_S08_S7_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/329.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide021_2019_09_10_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/77.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide009_2019_09_10_S19_S18_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/34.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide100_2019_09_10_S15_S14_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide003_2019_09_10_S11_S10_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/10.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide030_2019_09_10_S08_S7_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/113.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide101_2019_09_10_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/372.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide106_2019_09_10_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide053_2019_09_11_S22_S21_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/195.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide061_2019_09_11_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/223.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide024_2019_09_10_S01_S0_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/89.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_C0/DK23_slide004_2019_09_10_S06_S5_C1_TuYFP_raw.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail/

In [19]:
rfp = '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/'

rename_all_input_images(input_fp=rfp+'DK23_prep1_thumbnail_NtbNormalized', 
                        output_fp=rfp+'DK23_Neuroglancer_primary_channel_thumbnail', 
                        sorted_fns_fp=rfp+'DK23_sorted_filenames.txt')

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide035_2019_09_10_S15_S14_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/128.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide037_2019_09_10_S01_S0_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/133.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide109_2019_09_10_S19_S18_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/407.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide015_2019_09_10_S13_S12_C0_CFP_prep

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide094_2019_09_10_S22_S21_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/347.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide106_2019_09_10_S13_S12_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/394.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide003_2019_09_10_S06_S5_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/9.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide016_2019_09_10_S19_S18_C0_CFP_prep1_

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide032_2019_09_10_S01_S0_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/117.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide056_2019_09_11_S01_S0_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/204.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide047_2019_09_11_S22_S21_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/171.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide039_2019_09_10_S08_S7_C0_CFP_prep1_

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide005_2019_09_10_S06_S5_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/17.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide029_2019_09_10_S15_S14_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/110.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide110_2019_09_11_S19_S18_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/411.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide054_2019_09_03_S22_S21_C0_CFP_prep1

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide060_2019_09_11_S15_S14_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/221.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide009_2019_09_10_S13_S12_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/33.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide028_2019_09_10_S19_S18_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/107.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide075_2019_09_11_S01_S0_C0_CFP_prep1

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide083_2019_09_10_S01_S0_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/303.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide036_2019_09_10_S22_S21_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/132.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide029_2019_09_10_S08_S7_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/109.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide111_2019_09_10_S19_S18_C0_CFP_prep1

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide078_2019_09_11_S15_S14_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/285.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide096_2019_09_10_S15_S14_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/354.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide112_2019_09_11_S07_S6_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/417.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide025_2019_09_10_S01_S0_C0_CFP_prep1

cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide043_2019_09_11_S08_S7_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/154.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide017_2019_09_11_S07_S6_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/63.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide042_2019_09_11_S01_S0_C0_CFP_prep1_thumbnail_NtbNormalized.tif /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail/149.tif
cp /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_prep1_thumbnail_NtbNormalized/DK23_slide040_2019_09_10_S15_S14_C0_CFP_prep1_t

In [7]:
import os
import json

In [24]:
rfp = '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/'
precomputed_input_image_path = os.path.join( rfp, 'DK23_Neuroglancer_primary_channel_thumbnail_input')
precomputed_output_image_path = os.path.join( rfp, 'DK23_Neuroglancer_primary_channel_thumbnail_output')

data = {
    "ext": ".tif",
    "folder_to_convert_from": precomputed_input_image_path,
    "folder_to_convert_to": precomputed_output_image_path,
    "voxel_resolution": [460, 460, 20000],
    "voxel_offset": [0, 0, 0],
    "overwrite": True}
    
with open('input.json', 'w') as file:
    json.dump(data, file)

In [8]:
rfp = '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/'
precomputed_input_image_path = os.path.join( rfp, 'DK23_Neuroglancer_secondary_channel_thumbnail_input')
precomputed_output_image_path = os.path.join( rfp, 'DK23_Neuroglancer_secondary_channel_thumbnail_output')

data = {
    "ext": ".tif",
    "folder_to_convert_from": precomputed_input_image_path,
    "folder_to_convert_to": precomputed_output_image_path,
    "voxel_resolution": [460, 460, 20000],
    "voxel_offset": [0, 0, 0],
    "overwrite": True}
    
with open('input.json', 'w') as file:
    json.dump(data, file)

In [ ]:
neuroglancer.LocalVolume

In [2]:
# Filepaths to image files already on the computer
#    NOT NECESSARY as long as the image files are in the proper location
#    (check using `get_image_fp()`)
atlas_raw_jpeg_image_path = atlas_root+'CSHL_data_processed/'+stack+'/'+stack+'_prep2_raw_grayJpeg/'
atlas_sorted_fn_path = atlas_root+'CSHL_data_processed/'+stack+'/'+stack+'_sorted_filenames.txt'


# Input image files for precomputing script
temp_renamed_image_path = get_image_fp( stack, precomputed=False, brain_crop='brainstem', \
                                     image_version='grayJpeg', resolution='raw' )
precomputed_input_image_path = get_image_fp( stack, precomputed=False, brain_crop='brainstem', \
                                     image_version='grayJpeg', resolution='raw' )
temp_sorted_fn_path = precomputed_input_image_path+'../'+stack+'_sorted_filenames.txt'

# Output files from precomputing script
precomputed_output_image_path = get_image_fp( stack, precomputed=True, brain_crop='brainstem', \
                                     image_version='grayJpeg', resolution='raw' )

In [3]:
data = {
    "s3_creds_file": "/home/alexn/Documents/json_credentials/alex_aws_credentials.json",
    "s3_bucket_name_for_download": "mousebrainatlas-data",
    "sorted_filename": stack+"_sorted_filenames.txt",
    "ext": ".jpg",
    "folder_to_write_to": "/mnt/x/temp_s3_download/"+stack+"_prep2_lossless_grayJpeg",
    "folder_to_convert_from": precomputed_input_image_path,
    "folder_to_convert_to": precomputed_output_image_path,
    "s3_bucket_name_for_upload": "mousebrainatlas-datajoint-jp2k",
    "s3_dir_to_write_to": "precomputed/"+stack+"_fullres",
    "voxel_resolution": [460, 460, 20000],
    "voxel_offset": [0, 0, 0],
    "overwrite": True
}

with open('input.json', 'w') as file:
    json.dump(data, file)
  
 # RUN PRECOMPUTING SCRIPT
# running in terminal is much cleaner
#! python ../src/precomputation/precompute_images_v1.py input.json

In [13]:
rfp = '/data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/'
precomputed_input_image_path = os.path.join( rfp, 'DK23_Neuroglancer_primary_channel_thumbnail_input')
precomputed_output_image_path = os.path.join( rfp, 'DK23_Neuroglancer_primary_channel_thumbnail_output')

data = {
    "ext": ".tif",
    "folder_to_convert_from": precomputed_input_image_path,
    "folder_to_convert_to": precomputed_output_image_path,
    "voxel_resolution": [14720, 14720, 20000],
    "voxel_offset": [0, 0, 0],
    "overwrite": True}
    
with open('input.json', 'w') as file:
    json.dump(data, file)

In [11]:
460*32

14720

In [9]:
! pwd

/home/adnewber/neuroglancer_interface/notebooks


In [ ]:
images_local = data["folder_to_convert_from"]
precomputed_local = data["folder_to_convert_to"]
precomputed_s3 = 's3://mousebrainatlas-datajoint-jp2k/'+s3_precomputed_path_relative

with open( sorted_fn_path, 'r') as sfns:
    sorted_fn_list = sfns.read().split('\n')[:-1]
    
image_list = ! ls $images_local

In [4]:
# RUN PRECOMPUTING SCRIPT
# running in terminal is much cleaner
######! python neuroglancer_utilities.py input.json;
#! python ../src/precompute_images_v1.py input.json

In [5]:
# COPY PRECOMPUTED FILES TO S3
# running in terminal is much cleaner
#! aws s3 cp --recursive $precomputed_local $precomputed_s3

In [14]:
! aws s3 cp --recursive /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/ \
s3://neuroglancer-data/Tests/secondary/

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/118um/192-251_64-126_0-54 to s3://neuroglancer-data/Tests/secondary/118um/192-251_64-126_0-54
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/118um/192-251_0-64_0-54 to s3://neuroglancer-data/Tests/secondary/118um/192-251_0-64_0-54
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/118um/64-128_0-64_0-54 to s3://neuroglancer-data/Tests/secondary/118um/64-128_0-64_0-54
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/118um/64-128_64-126_0-54 to s3://neuroglancer-data/Tests/secondary/118um/64-128_64-126_0-54
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_second

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_320-384_128-192 to s3://neuroglancer-data/Tests/secondary/15um/0-64_320-384_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_320-384_256-320 to s3://neuroglancer-data/Tests/secondary/15um/0-64_320-384_256-320
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_320-384_192-256 to s3://neuroglancer-data/Tests/secondary/15um/0-64_320-384_192-256
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_320-384_384-425 to s3://neuroglancer-data/Tests/secondary/15um/0-64_320-384_384-425
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neurogl

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_64-128_192-256 to s3://neuroglancer-data/Tests/secondary/15um/0-64_64-128_192-256
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_64-128_320-384 to s3://neuroglancer-data/Tests/secondary/15um/0-64_64-128_320-384
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_64-128_64-128 to s3://neuroglancer-data/Tests/secondary/15um/0-64_64-128_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_640-704_0-64 to s3://neuroglancer-data/Tests/secondary/15um/0-64_640-704_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondar

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_960-1001_256-320 to s3://neuroglancer-data/Tests/secondary/15um/0-64_960-1001_256-320
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_960-1001_0-64 to s3://neuroglancer-data/Tests/secondary/15um/0-64_960-1001_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/0-64_960-1001_320-384 to s3://neuroglancer-data/Tests/secondary/15um/0-64_960-1001_320-384
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/1024-1088_0-64_128-192 to s3://neuroglancer-data/Tests/secondary/15um/1024-1088_0-64_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neu

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/1024-1088_320-384_64-128 to s3://neuroglancer-data/Tests/secondary/15um/1024-1088_320-384_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/1024-1088_384-448_128-192 to s3://neuroglancer-data/Tests/secondary/15um/1024-1088_384-448_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/1024-1088_320-384_256-320 to s3://neuroglancer-data/Tests/secondary/15um/1024-1088_320-384_256-320
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/15um/1024-1088_384-448_256-320 to s3://neuroglancer-data/Tests/secondary/15um/1024-1088_384-448_256-320
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing

...

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/768-832_128-192_0-64 to s3://neuroglancer-data/Tests/secondary/29um/768-832_128-192_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/768-832_320-384_128-192 to s3://neuroglancer-data/Tests/secondary/29um/768-832_320-384_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/768-832_320-384_0-64 to s3://neuroglancer-data/Tests/secondary/29um/768-832_320-384_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/768-832_384-448_192-213 to s3://neuroglancer-data/Tests/secondary/29um/768-832_384-448_192-213
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/832-896_320-384_64-128 to s3://neuroglancer-data/Tests/secondary/29um/832-896_320-384_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/832-896_256-320_64-128 to s3://neuroglancer-data/Tests/secondary/29um/832-896_256-320_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/832-896_384-448_128-192 to s3://neuroglancer-data/Tests/secondary/29um/832-896_384-448_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/832-896_384-448_64-128 to s3://neuroglancer-data/Tests/secondary/29um/832-896_384-448_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/896-960_384-448_64-128 to s3://neuroglancer-data/Tests/secondary/29um/896-960_384-448_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/896-960_448-501_0-64 to s3://neuroglancer-data/Tests/secondary/29um/896-960_448-501_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/896-960_64-128_192-213 to s3://neuroglancer-data/Tests/secondary/29um/896-960_64-128_192-213
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/896-960_448-501_128-192 to s3://neuroglancer-data/Tests/secondary/29um/896-960_448-501_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK2

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/960-1001_64-128_0-64 to s3://neuroglancer-data/Tests/secondary/29um/960-1001_64-128_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/960-1001_64-128_128-192 to s3://neuroglancer-data/Tests/secondary/29um/960-1001_64-128_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/960-1001_64-128_64-128 to s3://neuroglancer-data/Tests/secondary/29um/960-1001_64-128_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/29um/960-1001_64-128_192-213 to s3://neuroglancer-data/Tests/secondary/29um/960-1001_64-128_192-213
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/D

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/59um/320-384_192-251_0-64 to s3://neuroglancer-data/Tests/secondary/59um/320-384_192-251_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/59um/320-384_128-192_0-64 to s3://neuroglancer-data/Tests/secondary/59um/320-384_128-192_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/59um/256-320_64-128_64-107 to s3://neuroglancer-data/Tests/secondary/59um/256-320_64-128_64-107
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_secondary_channel_thumbnail_output/59um/320-384_64-128_0-64 to s3://neuroglancer-data/Tests/secondary/59um/320-384_64-128_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neurogl

In [15]:
! aws s3 cp --recursive /data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/ \
s3://neuroglancer-data/Tests/primary/

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/118um/64-128_64-126_0-54 to s3://neuroglancer-data/Tests/primary/118um/64-128_64-126_0-54
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/118um/192-251_64-126_0-54 to s3://neuroglancer-data/Tests/primary/118um/192-251_64-126_0-54
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/118um/192-251_0-64_0-54 to s3://neuroglancer-data/Tests/primary/118um/192-251_0-64_0-54
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/118um/0-64_0-64_0-54 to s3://neuroglancer-data/Tests/primary/118um/0-64_0-64_0-54
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_320-384_256-320 to s3://neuroglancer-data/Tests/primary/15um/0-64_320-384_256-320
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_320-384_64-128 to s3://neuroglancer-data/Tests/primary/15um/0-64_320-384_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_320-384_320-384 to s3://neuroglancer-data/Tests/primary/15um/0-64_320-384_320-384
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_320-384_384-425 to s3://neuroglancer-data/Tests/primary/15um/0-64_320-384_384-425
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_chan

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_64-128_192-256 to s3://neuroglancer-data/Tests/primary/15um/0-64_64-128_192-256
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_64-128_64-128 to s3://neuroglancer-data/Tests/primary/15um/0-64_64-128_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_64-128_0-64 to s3://neuroglancer-data/Tests/primary/15um/0-64_64-128_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_640-704_128-192 to s3://neuroglancer-data/Tests/primary/15um/0-64_640-704_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnai

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_960-1001_128-192 to s3://neuroglancer-data/Tests/primary/15um/0-64_960-1001_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_896-960_64-128 to s3://neuroglancer-data/Tests/primary/15um/0-64_896-960_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_960-1001_256-320 to s3://neuroglancer-data/Tests/primary/15um/0-64_960-1001_256-320
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/0-64_960-1001_192-256 to s3://neuroglancer-data/Tests/primary/15um/0-64_960-1001_192-256
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primar

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/1024-1088_320-384_128-192 to s3://neuroglancer-data/Tests/primary/15um/1024-1088_320-384_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/1024-1088_320-384_256-320 to s3://neuroglancer-data/Tests/primary/15um/1024-1088_320-384_256-320
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/1024-1088_320-384_320-384 to s3://neuroglancer-data/Tests/primary/15um/1024-1088_320-384_320-384
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/15um/1024-1088_320-384_192-256 to s3://neuroglancer-data/Tests/primary/15um/1024-1088_320-384_192-256
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_pro

...

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/704-768_64-128_64-128 to s3://neuroglancer-data/Tests/primary/29um/704-768_64-128_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/768-832_0-64_0-64 to s3://neuroglancer-data/Tests/primary/29um/768-832_0-64_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/768-832_0-64_192-213 to s3://neuroglancer-data/Tests/primary/29um/768-832_0-64_192-213
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/768-832_128-192_192-213 to s3://neuroglancer-data/Tests/primary/29um/768-832_128-192_192-213
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/896-960_192-256_0-64 to s3://neuroglancer-data/Tests/primary/29um/896-960_192-256_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/896-960_192-256_192-213 to s3://neuroglancer-data/Tests/primary/29um/896-960_192-256_192-213
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/896-960_128-192_64-128 to s3://neuroglancer-data/Tests/primary/29um/896-960_128-192_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/896-960_192-256_64-128 to s3://neuroglancer-data/Tests/primary/29um/896-960_192-256_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglan

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/960-1001_256-320_0-64 to s3://neuroglancer-data/Tests/primary/29um/960-1001_256-320_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/960-1001_256-320_64-128 to s3://neuroglancer-data/Tests/primary/29um/960-1001_256-320_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/960-1001_320-384_128-192 to s3://neuroglancer-data/Tests/primary/29um/960-1001_320-384_128-192
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/29um/960-1001_192-256_64-128 to s3://neuroglancer-data/Tests/primary/29um/960-1001_192-256_64-128
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_N

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/59um/192-256_128-192_0-64 to s3://neuroglancer-data/Tests/primary/59um/192-256_128-192_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/59um/192-256_128-192_64-107 to s3://neuroglancer-data/Tests/primary/59um/192-256_128-192_64-107
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/59um/192-256_64-128_64-107 to s3://neuroglancer-data/Tests/primary/59um/192-256_64-128_64-107
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/59um/192-256_192-251_64-107 to s3://neuroglancer-data/Tests/primary/59um/192-256_192-251_64-107
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_

upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/59um/64-128_0-64_64-107 to s3://neuroglancer-data/Tests/primary/59um/64-128_0-64_64-107
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/59um/64-128_0-64_0-64 to s3://neuroglancer-data/Tests/primary/59um/64-128_0-64_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/59um/64-128_128-192_0-64 to s3://neuroglancer-data/Tests/primary/59um/64-128_128-192_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_output/59um/64-128_192-251_0-64 to s3://neuroglancer-data/Tests/primary/59um/64-128_192-251_0-64
upload: ../../../../data2/Atlas_Root_Dirs/Beta_Testing/CSHL_data_processed/DK23/DK23_Neuroglancer_primary_channel_thumbnail_

In [3]:
import json

In [4]:
stack='MD661'

precomputed_input_image_path = '/nfs/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/\
CSHL_data_processed/MD661/MD661_prep2_raw_NtbNormalizedAdaptiveInvertedGammaJpeg/'
precomputed_output_image_path = '/nfs/birdstore/Active_Atlas_Data/copied_local_folders/MD661_precomputed/'

data = {
    "s3_creds_file": "/home/adnewber/Documents/json_credentials/alex_aws_credentials.json",
    "s3_bucket_name_for_download": "mousebrainatlas-data",
    "sorted_filename": stack+"_sorted_filenames.txt",
    "ext": ".jpg",
    "folder_to_write_to": "/mnt/x/temp_s3_download/"+stack+"_prep2_lossless_grayJpeg",
    "folder_to_convert_from": precomputed_input_image_path,
    "folder_to_convert_to": precomputed_output_image_path,
    "s3_bucket_name_for_upload": "mousebrainatlas-datajoint-jp2k",
    "s3_dir_to_write_to": "precomputed/"+stack+"_fullres",
    "voxel_resolution": [460, 460, 20000],
    "voxel_offset": [0, 0, 0],
    "overwrite": True
}

with open('input.json', 'w') as file:
    json.dump(data, file)
  
 # RUN PRECOMPUTING SCRIPT
# running in terminal is much cleaner
#! python ../src/precomputation/precompute_images_v1.py input.json

In [5]:
! python ../src/precomputation/precompute_images_v1.py input.json

============ Converting to precomputed ==============
Reading slices 0 to 7 (3.3 GiB memory needed)...                                
writing chunks:   6%|█▏                 | 392/6076 [00:02<00:39, 143.61chunks/s]